- 문자, 코드 포인트, 바이트 표현
- 이진 시퀀스의 고유한 특징 : bytes, bytearray, memoryview
- 전체 유니코드 및 레거시 문자셋에 대한 코덱
- 인코딩 에러를 피하고 다루기
- 텍스트 파일을 다룰 때의 모범 사례
- 기본 인코딩 및 표준 입출력 문제
- 정규화를 이용한 안전한 유니코드 텍스트 비교
- 정규화, 케이스 폴딩, 발음 구별 기호 강제 제거를 위한 유틸리티 함수
- locale과 PyUCA 라이브러리를 이용한 유니코드 텍스트의 적절한 정렬
- 유니코드 데이터베이스 안의 문자 메타데이터
- str과 bytes를 다루는 이중모드 API

### 4.1 문자 문제
- 코드 포인트를 바이트로 변환하는 것을 __인코딩__, 바이트를 코드 포인트로 변환하는 것을 __디코딩__ 이라고 한다.

In [9]:
s = 'café'
print (len(s))
b= s.encode('utf8')
print (b)
len(b)
print (b.decode('utf8'))

4
b'caf\xc3\xa9'
café


### 4.2 바이트에 대한 기본 지식

In [21]:
cafe = bytes ('café', encoding='utf8')
print (cafe)
print (cafe[0])
print (cafe[:1])
cafe_arr =  bytearray(cafe)
print(cafe_arr)
print (cafe_arr[-1:])

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


- 화면에 출력 가능한 아스키 문자(공백에서 물결표(~)까지)는 아스키 문자 그대로 출력한다.
- 탭, 개행 문자, 캐리지 리턴, 백슬래시(\)는 이스케이프 시퀀스(\t, \n, \r, \\\\)로 출력한다.
- 그 외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력한다.

In [22]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [23]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
print (octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


#### 4.2.1 구조체와 메모리 뷰

In [26]:
import struct
fmt = "<3s3sHH"
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())
header = img[:10]
print (bytes(header))
struct.unpack(fmt, header)
del header
del img

FileNotFoundError: [Errno 2] No such file or directory: 'filter.gif'

### 4.3 기본 인코더/디코더

In [45]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep="\t")
    

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


### 4.4 인코딩/디코딩 문제 이해하기
#### 4.4.1 UnicodeEoncdeError 처리하기

In [46]:
city = 'São Paulo'
print (city.encode('utf-8'))
print (city.encode('utf-16'))
print (city.encode('iso8859_1'))
#print (city.encode('cp437'))
print (city.encode('cp437', errors = 'ignore'))
print (city.encode('cp437', errors = 'replace'))
print (city.encode('cp437', errors = 'xmlcharrefreplace'))

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'
b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


#### 4.4.2 UnicodeDecodeError 처리하기

In [47]:
octets = b'Montr\xe9al'
print (octets.decode("cp1252"))
print (octets.decode("iso8859_7"))
print (octets.decode("koi8_r"))
#print (octets.decode("utf8")) 
print (octets.decode("utf8", errors='replace')) 

Montréal
Montrιal
MontrИal
Montr�al


#### 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError
#### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법
#### 4.4.5 BOM : 유용한 깨진 문자

In [52]:
u16 = 'El Niño'.encode('utf_16')
print (u16)
print (list(u16))
u16le = 'El Niño'.encode('utf_16le')
print (list(u16le))

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'
[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
